In [ ]:
%%sh

dc="SERVER PREFIX GOES HERE"
apikey="API KEY GOES HERE"

curl -sS \
  "https://${dc}.api.mailchimp.com/3.0/ping" \
  --user "anystring:${apikey}" | jq -r

In [ ]:
f = open("reports.json", "w")

In [ ]:
%%sh

dc="SERVER PREFIX GOES HERE"
apikey="API KEY GOES HERE"

curl -X GET \
  "https://${dc}.api.mailchimp.com/3.0/reports?&count=100&offset=0" \
  --user "anystring:${apikey}" -o reports.json

In [ ]:
import json

with open('reports.json') as reports_file:
    data = json.load(reports_file)

    print("Type:", type(data))

In [ ]:
import pandas as pd
def data_to_pandas():
    
    df = pd.DataFrame(columns=['id', 'send_time','campaign_title', 'type', 'list_name', 'subject_line', 
                               'preview_text', 'emails_sent', 'abuse_reports', 'unsubscribed', 
                               'hard_bounces', 'soft_bounces', 'syntax_errors', 'forwards_count',
                               'forwards_opens', 'opens_total', 'unique_opens', 'open_rate',
                               'clicks_total', 'unique_clicks', 'unique_subscriber_clicks', 
                               'click_rate', 'list_sub_rate', 'list_unsub_rate', 'list_open_rate', 
                               'list_click_rate', 'total_orders', 'total_revenue'
                              ])
    
    if data:
        for report in data['reports']: 
            row = {
                'id': report.get('id'),
                'send_time': report.get('send_time'),                
                'campaign_title': report.get('campaign_title'),
                'type': report.get('type'),
                'list_name': report.get('list_name'),
                'subject_line': report.get('subject_line'),
                'preview_text': report.get('preview_text'),
                'emails_sent': report.get('emails_sent'),
                'abuse_reports': report.get('abuse_reports'),
                'unsubscribed': report.get('unsubscribed'),
                'hard_bounces': report.get('bounces').get('hard_bounces'),
                'soft_bounces': report.get('bounces').get('soft_bounces'),
                'syntax_errors': report.get('syntax_errors'),
                'forwards_count': report.get('forwards').get('forwards_count'),
                'forwards_opens': report.get('forwards').get('forwards_opens'),
                'opens_total': report.get('opens').get('opens_total'),
                'unique_opens': report.get('opens').get('unique_opens'),
                'open_rate': report.get('opens').get('open_rate'),
                'clicks_total': report.get('clicks').get('clicks_total'),
                'unique_clicks': report.get('clicks').get('unique_clicks'),
                'unique_subscriber_clicks': report.get('clicks').get('unique_subscriber_clicks'),
                'click_rate': report.get('clicks').get('click_rate'),
                'list_sub_rate': report.get('list_stats').get('sub_rate'),
                'list_unsub_rate': report.get('list_stats').get('unsub_rate'),
                'list_open_rate': report.get('list_stats').get('open_rate'),
                'list_click_rate': report.get('list_stats').get('click_rate'),
                'total_orders': report.get('ecommerce').get('total_orders'),
                'total_revenue': report.get('ecommerce').get('total_revenue'),
            }
            
            df = df.append(row, ignore_index='True')
        
        df = df.fillna(0)
            
        for col in ['emails_sent', 'abuse_reports', 'unsubscribed', 
                    'hard_bounces', 'soft_bounces', 'syntax_errors',
                    'forwards_count', 'forwards_opens', 'opens_total', 
                    'unique_opens', 'open_rate', 'clicks_total', 
                    'unique_clicks', 'unique_subscriber_clicks', 'click_rate',
                    'list_sub_rate', 'list_unsub_rate', 'list_unsub_rate', 
                    'list_open_rate', 'list_click_rate', 'total_orders']:
            df[col] = df[col].astype(int)
            
        df['total_revenue'] = df['total_revenue'].astype(float)
        
    return df

In [ ]:
df = data_to_pandas()
# df = df.loc[df['list_name'] == "YOUR AUDIENCE NAME"].reset_index(drop=True)
df.head()

In [ ]:
import datetime as dt
df['send_time'] = pd.to_datetime(df['send_time'], format="%Y-%m-%dT%H:%M:%S.%f").dt.date
display(df)

In [ ]:
per = df[["send_time","subject_line", "emails_sent", "unique_opens", 
                          "unique_subscriber_clicks", "unsubscribed","soft_bounces","hard_bounces"]].copy()
per.head()

In [ ]:
facts = pd.DataFrame([per["send_time"], per["subject_line"], per["emails_sent"]]).copy().transpose()
facts.head()

In [ ]:
kpis = {"OR": per["unique_opens"]/per["emails_sent"],
           "CTR": per["unique_subscriber_clicks"]/per["emails_sent"],
           "CTOR": per["unique_subscriber_clicks"]/per["unique_opens"],
           "SBR": per["soft_bounces"]/per["emails_sent"],
           "HBR": per["hard_bounces"]/per["emails_sent"],
           "UR": per["unsubscribed"]/per["emails_sent"]}
kpis = pd.DataFrame(kpis)
pd.options.display.float_format = '{:.1%}'.format
kpis.head()

In [ ]:
mr = pd.concat([facts, kpis], axis=1)
mr.set_index('send_time')

In [ ]:
filename = 'mc_report_'+str(dt.datetime.now().date())+'.xlsx'
mr.to_excel(filename, sheet_name='data')
print(filename)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(23,10))

date = mr["send_time"]
open_rate = mr["OR"]
click_to_open_rate = mr["CTOR"]
click_rate = mr["CTR"]
unsubscribe_rate = mr["UR"]

ax1 = plt.plot(date, open_rate, color='red', marker='o', label="open rate")
ax2 = plt.plot(date, click_to_open_rate, color='blue', marker='o', label="click to open rate")
ax3 = plt.plot(date, click_rate, color='green', marker='o', label="click rate")
ax4 = plt.plot(date, unsubscribe_rate, color='grey', marker='o', label="unsubscribe rate")

plt.title('Email Campaign Performance Chart', fontsize=20)
plt.xlabel('Email Send Date', fontsize=16)
plt.ylabel('Performance Rate (%)', fontsize=16)
plt.legend()
plt.grid(True)

plt.show()